In [ ]:
# |default_exp rss

In [ ]:
from config import get_RSS_URL

get_RSS_URL()

'https://radiofrance-podcast.net/podcast09/rss_14007.xml'

# extraire_urls

In [ ]:
# |export

import feedparser
import re
from typing import List
from config import get_RSS_URL


def extraire_dureesummary(summary: str) -> int:
    """
    Extrait la durée d'un épisode du masque.

    Args:
        summary (str): Le résumé de l'épisode contenant la durée.

    Returns:
        int: Le nombre de secondes correspondant à la durée d'un épisode.
        Retourne -1 si la durée n'est pas trouvée.
    """
    # Expression régulière pour extraire la durée
    pattern_duree = r"durée\s*:\s*(\d{2}:\d{2}:\d{2})"

    # Recherche de la durée dans le texte
    match = re.search(pattern_duree, summary)

    if match:
        duree_str = match.group(1)
        heures, minutes, secondes = map(int, duree_str.split(":"))
        return heures * 3600 + minutes * 60 + secondes
    else:
        return -1


def extraire_urls_rss(duree_mini_minutes: int = 15) -> List[str]:
    """
    Extrait les URLs des balises `enclosure` d'un flux RSS des épisodes durant plus de `duree_mini_minutes` minutes.

    Args:
        duree_mini_minutes (int): La durée minimale en minutes des épisodes du flux.

    Returns:
        List[str]: Une liste d'URLs.
    """
    url_flux = get_RSS_URL()

    flux = feedparser.parse(url_flux)
    urls = []
    for entree in flux.entries:
        for link in entree.links:
            if link.type == "audio/mpeg":
                if extraire_dureesummary(entree.summary) > duree_mini_minutes * 60:
                    urls.append(link.href)
    return urls

In [ ]:
urls = extraire_urls_rss()
urls

['https://rf.proxycast.org/87a61bc1-2fa4-4f27-9eee-521c54c34844/14007-09.02.2025-ITEMA_24023310-2025F4007S0040-NET_MFI_3BC0392B-ACF3-4C53-8749-67830363DCB0-22.mp3',
 'https://rf.proxycast.org/ac4e31c0-fe10-40d0-83ce-d52cd6b370e9/14007-02.02.2025-ITEMA_24015816-2025F4007S0033-NET_MFI_86F6DBE0-8737-48DC-A1A4-8BF55BF89BC1-22.mp3',
 'https://rf.proxycast.org/f421bbf9-5e6e-4411-85a6-7c6a318d2073/14007-26.01.2025-ITEMA_24008377-2025F4007S0026-NET_MFI_1EBC23A2-31A4-4066-BC10-C8067ADD259C-22.mp3',
 'https://rf.proxycast.org/881d7c53-ef7e-44bd-a5b4-e6e3bbe6aa58/14007-19.01.2025-ITEMA_24000708-2025F4007S0019-22.mp3',
 'https://rf.proxycast.org/ad97aa2e-ebfc-4d00-8739-4ca72192e726/14007-12.01.2025-ITEMA_23993269-2025F4007S0012-22.mp3',
 'https://rf.proxycast.org/a3082fcd-8ed2-49eb-883c-29b32a1f7b2b/14007-05.01.2025-ITEMA_23985183-2025F4007S0005-22.mp3',
 'https://rf.proxycast.org/a9ea02c4-09a0-4a95-a86b-269c571baf8a/14007-29.12.2024-ITEMA_23978947-2024F4007S0364-22.mp3',
 'https://rf.proxycast.or

# podcast

In [ ]:
# |export

import feedparser
from mongo import get_collection, get_DB_VARS
from datetime import datetime
from typing import List, Optional
from feedparser.util import FeedParserDict
from mongo_episode import RSS_episode
import pytz

RSS_DATE_FORMAT = "%a, %d %b %Y %H:%M:%S %z"  # "Sun, 29 Dec 2024 10:59:39 +0100"


class Podcast:
    def __init__(self):
        """
        Initialise la classe Podcast en analysant le flux RSS et en obtenant la collection MongoDB.
        """
        self.parsed_flow = feedparser.parse(get_RSS_URL())
        DB_HOST, DB_NAME, _ = get_DB_VARS()
        self.collection = get_collection(
            target_db=DB_HOST, client_name=DB_NAME, collection_name="episodes"
        )

    def get_most_recent_episode_from_DB(self) -> Optional[datetime]:
        """
        Récupère la date la plus récente des épisodes stockés dans la base de données.

        Returns:
            Optional[datetime]: La date la plus récente des épisodes stockés, ou None si aucun épisode n'est trouvé.
        """
        most_recent_document = self.collection.find().sort({"date": -1}).limit(1)
        most_recent_date = None
        for doc in most_recent_document:
            most_recent_date = doc["date"].replace(tzinfo=pytz.timezone("Europe/Paris"))
        return most_recent_date

    def list_last_large_episodes(
        self, duree_mini_minutes: int = 15
    ) -> List[FeedParserDict]:
        """
        Liste les épisodes RSS qui sont plus récents que le plus récent épisode stocké dans la base de données
        et qui durent plus de `duree_mini_minutes` minutes.

        Args:
            duree_mini_minutes (int): La durée minimale en minutes des épisodes à lister. Par défaut à 15 minutes.

        Returns:
            List[FeedParserDict]: Une liste d'entrées RSS correspondant aux critères.
        """
        last_large_episodes = []
        for entry in self.parsed_flow.entries:
            date_rss = datetime.strptime(entry.published, RSS_DATE_FORMAT)
            date_db = self.get_most_recent_episode_from_DB()
            if date_db and date_rss > date_db:
                if (
                    RSS_episode.get_duree_in_seconds(entry.itunes_duration)
                    > duree_mini_minutes * 60
                ):
                    last_large_episodes.append(entry)
        return last_large_episodes

    def store_last_large_episodes(self, duree_mini_minutes: int = 15) -> None:
        """
        Parcourt la liste des épisodes longs récents, instancie RSS_episode et les conserve dans la base de données.
        Affiche le nombre de mises à jour réussies dans la base de données.

        Args:
            duree_mini_minutes (int): La durée minimale en minutes des épisodes à stocker. Par défaut à 15 minutes.
        """
        updates = 0
        last_large_episodes = self.list_last_large_episodes(duree_mini_minutes)
        for entry in last_large_episodes:
            rss_entry = RSS_episode.from_feed_entry(entry)
            updates += rss_entry.keep()
        print(f"Updated episodes: {updates}")

In [ ]:
podcast = Podcast()
podcast.get_most_recent_episode_from_DB()

datetime.datetime(2025, 1, 26, 10, 59, 39, tzinfo=<DstTzInfo 'Europe/Paris' LMT+0:09:00 STD>)

In [ ]:
len(podcast.list_last_large_episodes())

2

In [ ]:
podcast.store_last_large_episodes()

Device set to use cpu


Episode du 09 Feb 2025 10:59 ignored: Duree: 2900, Type: films


Device set to use cpu


Episode du 02 Feb 2025 10:59 ignored: Duree: 2898, Type: films
Updated episodes: 0


# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("py rss helper.ipynb", ".")